In [1]:
import os

import numpy as np
from sklearn.linear_model import LogisticRegression

from utils import load_hidden_representations_from_hdf5, read_templates_from_file

----

In [2]:
# params
log_dir = "/logfiles"
model = "bigscience-T0"
module = "encoder"
task = "rte"

In [3]:
# assertions
assert model == "bigscience-T0"
assert module == "encoder"
assert task == "rte"

## Prompts

In [4]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}/all.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_prompt.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_target_yes_no.csv")
display(df)

,name,template,category,includes_targets,targets,target_ids,shuffle
0,gpt_3_yes_no_with_targets,{premise} Question: {hypothesis} Yes or No?,instructive,True,"▁Yes, ▁No","0, 1",False
1,mnli_crowdsource_with_targets,{premise} Using only the above description and...,instructive,True,"▁Yes, ▁No","0, 1",False
2,based_on_previous_passage_with_targets,"{premise} Based on the previous passage, is it...",instructive,True,"▁Yes, ▁No","0, 1",False
3,infer_with_targets,"Suppose {premise} Can we infer that ""{hypothes...",instructive,True,"▁Yes, ▁No","0, 1",False
4,follow_with_targets,Given that {premise} Does it follow that {hypo...,instructive,True,"▁Yes, ▁No","0, 1",False
5,imply_with_targets,"{premise} Question: Does this imply that ""{hyp...",instructive,True,"▁Yes, ▁No","0, 1",False
6,guaranteed_with_targets,"Given {premise} Is it guaranteed true that ""{h...",instructive,True,"▁Yes, ▁No","0, 1",False
7,justified_with_targets,"{premise} Are we justified in saying that ""{hy...",instructive,True,"▁Yes, ▁No","0, 1",False
8,must_be_true_with_targets,"Given that {premise} Therefore, it must be tru...",instructive,True,"▁Yes, ▁No","0, 1",False
9,should_assume_with_targets,"Given {premise} Should we assume that ""{hypoth...",instructive,True,"▁Yes, ▁No","0, 1",False


## Train linear classifier

In [5]:
def unison_shuffled_copies(a, b):
    # from: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [6]:
use_pattern = [
    "gpt_3_yes_no_with_targets",
    # "mnli_crowdsource_with_targets",
    # "based_on_previous_passage_with_targets",
    # "infer_with_targets",
    # "follow_with_targets",
    # "imply_with_targets",
    # "guaranteed_with_targets",
    # "justified_with_targets",
    # "must_be_true_with_targets",
    # "should_assume_with_targets",
    "gpt_3_true_false_with_targets",
    # "gpt_3_cat_dog_with_targets",
    # "gpt_3_yes_no_without_targets",
]

In [7]:
# for layer in range(0, 10):
for layer in range(0, 25):
# for layer in range(24, 25):
    
    print("layer:", layer)
    file_names, prompt_names = [], []

    for _, row in df.iterrows():
        if row['name'] in use_pattern:
            file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg.hdf5",)
            prompt_names.append(row['name'])

    # load hidden representations from hdf5 file
    representations = None
    classes = []
    n_sequences = 0

    for idx, file_name in enumerate(file_names):
        hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
        # print(hidden_representations.shape)
        n_sequences = hidden_representations.shape[0]

        if representations is None:
            representations = hidden_representations
        else:
            representations = np.concatenate((representations, hidden_representations), axis=0)

        classes += n_sequences * [idx] # assign representations to classes
    
    classes = np.asarray(classes)

    # shuffle representations and classes
    X, y = unison_shuffled_copies(representations, classes)
    print(X.shape, y.shape)

    # train linear classifier
    # multi_class='multinomial' uses a CE loss
    print('classifying between:', prompt_names)
    clf = LogisticRegression(random_state=0, max_iter=2000, multi_class='multinomial').fit(X, y)
    
    print(f'layer={layer}; accuracy on training data: ', clf.score(X, y))
    print('\n')



layer: 0


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5862.55it/s]

(554, 4096) (554,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']


layer=0; accuracy on training data:  1.0


layer: 1


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5566.84it/s]

...

(554, 4096) (554,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']


layer=23; accuracy on training data:  1.0


layer: 24


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5572.21it/s]

(554, 4096) (554,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']


layer=24; accuracy on training data:  0.9963898916967509


